In [5]:
from TTS.api import TTS
tts_2 = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

/home/ahmet/anaconda3/envs/tts_env/lib/python3.10/site-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.


/home/ahmet/anaconda3/envs/tts_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 > Using model: xtts


/home/ahmet/anaconda3/envs/tts_env/lib/python3.10/site-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.lo

In [3]:
import librosa
import soundfile as sf

def resample_wav(input_wav, output_wav, target_sr=44100):
    """
    Resample a WAV file to the target sample rate.

    :param input_wav: Path to the input WAV file (e.g., "output_22k.wav")
    :param output_wav: Path to the resampled output WAV file (e.g., "output_44k.wav")
    :param target_sr: Target sample rate (default: 44100Hz)
    """
    # Load audio file
    audio, sr = librosa.load(input_wav, sr=None)  # Load with original SR

    # Resample audio
    resampled_audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)

    # Save the resampled audio
    sf.write(output_wav, resampled_audio, target_sr)

    print(f"✅ Resampled {input_wav} from {sr}Hz to {target_sr}Hz → Saved as {output_wav}")

# Example usage after cloning:
# Assuming cloned output is saved as "output_22k.wav"



In [22]:
REFERENCE_AUDIO = "audio_dataset/wavs/segment_001.wav"
sentence = "Manchmal ist es besser, nur die für den Sprecher relevanten Ebenen zu optimieren"

tts_2.tts_to_file(text=sentence, speaker_wav=REFERENCE_AUDIO, language="de", file_path="de_org.wav")
resample_wav("de_org.wav", "de_org_44.wav", target_sr=44100)

 > Text splitted to sentences.
['Manchmal ist es besser, nur die für den Sprecher relevanten Ebenen zu optimieren']
 > Processing time: 1.0737414360046387
 > Real-time factor: 0.20058965927801176
✅ Resampled de_org.wav from 24000Hz to 44100Hz → Saved as de_org_44.wav


In [23]:
import torch
from TTS.api import TTS
import torch
torch.cuda.empty_cache()  # ✅ Clears GPU memory cache
# ✅ Define paths
MODEL_PATH = "fine_tuning_output/XTTS_v2_FT"
CONFIG_PATH = "fine_tuning_output/XTTS_v2_FT/config.json"
REFERENCE_AUDIO= "audio_dataset/wavs/segment_001.wav"
# ✅ Load fine-tuned XTTS
tts = TTS(model_path=MODEL_PATH, config_path=CONFIG_PATH).to("cuda")

# ✅ Define test sentence
sentence = "Manchmal ist es besser, nur die für den Sprecher relevanten Ebenen zu optimieren"

# ✅ Generate speech
tts.tts_to_file(text=sentence, speaker_wav=REFERENCE_AUDIO, language="de", file_path="de_FT.wav")
resample_wav("de_FT.wav", "de_FT_44.wav", target_sr=44100)
print("🎉 Fine-tuned voice generated! Check output.wav")


 > Using model: xtts
 > Text splitted to sentences.
['Manchmal ist es besser, nur die für den Sprecher relevanten Ebenen zu optimieren']
 > Processing time: 0.9570026397705078
 > Real-time factor: 0.18946548813873454
✅ Resampled de_FT.wav from 22050Hz to 44100Hz → Saved as de_FT_44.wav
🎉 Fine-tuned voice generated! Check output.wav


In [25]:
import librosa

def get_wav_sample_rate(wav_path):
    """Returns the sample rate (Hz) of a given WAV file."""
    try:
        sample_rate = librosa.get_samplerate(wav_path)
        return sample_rate
    except Exception as e:
        return f"Error: {e}"

# Example usage (update with your actual file path)
wav_file_path = "output_44_2.wav"  # Replace with your actual file
sample_rate = get_wav_sample_rate(wav_file_path)
sample_rate


44100

In [39]:
from phonemizer import phonemize

word = "nasilsin"
ipa = phonemize(word, language="tr", backend="espeak")
print(ipa)  # Output: /teʃeˈkʰyɾ/


nasɪlsɪn 


In [7]:
# ✅ Define Turkish → IPA Mapping
IPA_DICT = {
    "a": "a", "b": "b", "c": "dʒ", "ç": "tʃ", "d": "d", "e": "e", "f": "f",
    "g": "ɡ", "ğ": "ɣ", "h": "h", "ı": "ɯ", "i": "i", "j": "ʒ", "k": "k",
    "l": "l", "m": "m", "n": "n", "o": "o", "ö": "ø", "p": "p", "r": "ɾ",
    "s": "s", "ş": "ʃ", "t": "t", "u": "u", "ü": "y", "v": "v", "y": "j", "z": "z",
    # Special cases for better pronunciation
    "ev": "ev", "an": "ɑn", "en": "en", "at": "ɑt", "et": "et", 
    "il": "il", "ol": "ol", "ul": "ul", "el": "el",
    "ay": "aj", "ey": "ej", "oy": "oj", "uy": "uj"
}

# ✅ Function to Convert Turkish Text → IPA
def turkish_to_ipa(word):
    """
    Converts a Turkish word into IPA representation.
    """
    word = word.lower()  # Ensure lowercase
    ipa_word = ""
    
    for char in word:
        ipa_word += IPA_DICT.get(char, char)  # Replace using dictionary
    
    return f"/{ipa_word}/"  # Return IPA transcription

# ✅ Test the function
words = ["merhaba", "nasılsın", "teşekkür", "güzel", "Türkçe", "kedi", "şişe", "uçak"]
words = "seni gördügümde gercekten mutlu oluyorum. inmazsan sor".split(" ")
ipa_transcriptions = {word: turkish_to_ipa(word) for word in words}

# ✅ Display results
import pandas as pd
df = pd.DataFrame(ipa_transcriptions.items(), columns=["Turkish", "IPA"])

print(df.to_string(index=False))


   Turkish          IPA
      seni       /seni/
gördügümde /ɡøɾdyɡymde/
 gercekten /ɡeɾdʒekten/
     mutlu      /mutlu/
 oluyorum.  /olujoɾum./
  inmazsan   /inmazsan/
       sor        /soɾ/


In [44]:
import re

# ✅ Turkish letter → IPA phoneme mapping
turkish_to_ipa_dict = {
    "a": "a", "b": "b", "c": "dʒ", "ç": "tʃ", "d": "d",
    "e": "ɛ", "f": "f", "g": "ɡ", "ğ": "ɣ", "h": "h",
    "ı": "ɯ", "i": "i", "j": "ʒ", "k": "k", "l": "ɫ",
    "m": "m", "n": "n", "o": "o", "ö": "ø", "p": "p",
    "r": "ɾ", "s": "s", "ş": "ʃ", "t": "t", "u": "u",
    "ü": "y", "v": "v", "y": "j", "z": "z"
}

# ✅ Function to divide words into syllables
def syllabify(word):
    vowels = "aıoueiöü"
    syllables = []
    current_syllable = ""

    for i, char in enumerate(word):
        current_syllable += char
        if char in vowels:
            if i < len(word) - 1 and word[i + 1] not in vowels:
                syllables.append(current_syllable)
                current_syllable = ""
    if current_syllable:
        syllables.append(current_syllable)

    return syllables

# ✅ Function to place primary stress correctly
def add_primary_stress(syllables):
    if len(syllables) == 1:
        return syllables  # One-syllable words don’t need stress
    
    # Default: Stress the **last vowel-ending syllable**
    for i in range(len(syllables) - 1, -1, -1):
        if syllables[i][-1] in "aıoueiöü":
            syllables[i] = syllables[i] + "ˈ"
            break

    return syllables

# ✅ Function to convert Turkish text to IPA
def turkish_to_ipa(word):
    word = word.lower()
    
    # Step 1: Convert to IPA
    ipa_word = "".join([turkish_to_ipa_dict.get(char, char) for char in word])

    # Step 2: Syllabify
    syllables = syllabify(ipa_word)

    # Step 3: Add correct stress placement
    stressed_syllables = add_primary_stress(syllables)

    return "/{}/".format(".".join(stressed_syllables))

# ✅ Example words
words = ["merhaba", "nasılsın", "teşekkür", "güzel", "doktor", "telefon", "kaplan", "yapıyor", "yapma", "evde"]

# ✅ Convert words to IPA
ipa_results = {word: turkish_to_ipa(word) for word in words}

# ✅ Display Results
for word, ipa in ipa_results.items():
    print(f"{word} → {ipa}")


merhaba → /mɛɾha.baˈ/
nasılsın → /naˈ.sɯɫsɯn/
teşekkür → /tɛʃɛkkyɾ/
güzel → /ɡyzɛɫ/
doktor → /do.ktoˈ.ɾ/
telefon → /tɛɫɛfoˈ.n/
kaplan → /ka.pɫaˈ.n/
yapıyor → /ja.pɯjoˈ.ɾ/
yapma → /ja.pmaˈ/
evde → /ɛvdɛ/


In [47]:
def get_syllables(word):
    syllables = []

    """
    Aşağıdaki satır gelen kelimenin ünlü harfler 1, ünsüzler 0 olacak
    şekilde desenini çıkarır.
    Örneğin: arabacı -> 1010101, türkiye -> 010010
    """

    bits = ''.join(['1' if l in 'aeıioöuü' else '0' for l in word])

    """
    Aşağıdaki seperators listesi, yakalanacak desenleri ve desen yakalandığında
    kelimenin hangi pozisyondan kesileceğini tanımlıyor.
    Türkçede kelime içinde iki ünlü arasındaki ünsüz, kendinden sonraki
    ünlüyle hece kurar., yani 101 desenini yakaladığımızda kelimeyi
    bulunduğumuz yerden 1 ileri pozisyondan kesmeliyiz. ('101', 1)
    Kelime içinde yan yana gelen iki ünsüzden ilki kendinden önceki ünlüyle,
    ikincisi kendinden sonraki ünlüyle hece kurar. Bu da demek oluyor ki
    1001 desenini yakaladığımızda kelimeyi bulunduğumuz noktadan 2 ileriden
    kesmeliyiz. ('1001', 2),
    Kelime içinde yan yana gelen üç ünsüz harften ilk ikisi kendinden önceki
    ünlüyle, üçüncüsü kendinden sonraki ünlüyle hece kurar. Yani 10001 desenini
    gördüğümüzde kelimeyi bulunduğumuz yerden 3 ileri pozisyondan kesmemiz
    gerek. ('10001', 3)
    """

    seperators = (
        ('101', 1),
        ('1001', 2),
        ('10001', 3)
    )

    index, cut_start_pos = 0, 0

    # index değerini elimizdeki bitler üzerinde yürütmeye başlıyoruz.
    while index < len(bits):

        """
        Elimizdeki her ayırıcıyı (seperator), bits'in index'inci karakterinden
        itibarent tek tek deneyerek yakalamaya çalışıyoruz.
        """

        for seperator_pattern, seperator_cut_pos in seperators:
            if bits[index:].startswith(seperator_pattern):

                """
                Yakaladığımızda, en son cut_start posizyonundan, bulunduğumuz
                pozisyonun serpator_cut_pos kadar ilerisine kadar bölümü alıp
                syllables sepetine atıyoruz.
                """

                syllables.append(word[cut_start_pos:index + seperator_cut_pos])

                """
                Index'imiz seperator_cut_pos kadar ilerliyor, ve
                cut_start_pos'u index'le aynı yapıyoruz.
                """

                index += seperator_cut_pos
                cut_start_pos = index
                break

        """
        Index ilerliyor, cut_start_pos'da değişiklik yok.
        """

        index += 1

    # Son kalan heceyi elle sepete atıyoruz.
    syllables.append(word[cut_start_pos:])
    return syllables

print(get_syllables(u'araba'))
print(get_syllables(u'biçimine'))
print(get_syllables(u'insanın'))
print(get_syllables(u'karaca'))

print(get_syllables(u'aldı'))
print(get_syllables(u'birlik'))
print(get_syllables(u'sevmek'))

print(get_syllables(u'altlık'))
print(get_syllables(u'türkçe'))
print(get_syllables(u'korkmak'))
print(get_syllables(u'altini oymak'))

['a', 'ra', 'ba']
['bi', 'çi', 'mi', 'ne']
['in', 'sa', 'nın']
['ka', 'ra', 'ca']
['al', 'dı']
['bir', 'lik']
['sev', 'mek']
['alt', 'lık']
['türk', 'çe']
['kork', 'mak']
['al', 'ti', 'ni', ' oy', 'mak']


In [10]:
import re

# ✅ Turkish letter → IPA phoneme mapping
turkish_to_ipa_dict = {
    "a": "a", "b": "b", "c": "dʒ", "ç": "tʃ", "d": "d",
    "e": "ɛ", "f": "f", "g": "ɡ", "ğ": "ɣ", "h": "h",
    "ı": "ɯ", "i": "i", "j": "ʒ", "k": "k", "l": "ɫ",
    "m": "m", "n": "n", "o": "o", "ö": "ø", "p": "p",
    "r": "ɾ", "s": "s", "ş": "ʃ", "t": "t", "u": "u",
    "ü": "y", "v": "v", "y": "j", "z": "z"
}

# ✅ Your syllabification function
def get_syllables(word):
    vowels = "aeıioöuü"
    bits = ''.join(['1' if l in vowels else '0' for l in word])

    seperators = [
        ('101', 1),
        ('1001', 2),
        ('10001', 3)
    ]

    index, cut_start_pos = 0, 0
    syllables = []

    while index < len(bits):
        for pattern, cut_pos in seperators:
            if bits[index:].startswith(pattern):
                syllables.append(word[cut_start_pos:index + cut_pos])
                index += cut_pos
                cut_start_pos = index
                break
        index += 1

    syllables.append(word[cut_start_pos:])
    return syllables

# ✅ Function to add primary stress
def add_primary_stress(syllables):
    if len(syllables) == 1:
        return syllables  # One-syllable words don’t need stress
    syllables[-1] = "ˈ" + syllables[-1]  # Default: stress last syllable
    return syllables

# ✅ Convert Turkish → IPA with syllables
def turkish_to_ipa(word):
    word = word.lower()
    
    # Step 1: Get syllables
    syllables = get_syllables(word)

    # Step 2: Convert each syllable to IPA
    ipa_syllables = []
    for syl in syllables:
        ipa_syll = "".join([turkish_to_ipa_dict.get(char, char) for char in syl])
        ipa_syllables.append(ipa_syll)

    # Step 3: Add stress
    stressed_syllables = add_primary_stress(ipa_syllables)

    return "/{}/".format(".".join(stressed_syllables))

# ✅ Example words
words = ["değil","merhaba", "nasılsın", "teşekkür", "güzel", "doktor", "telefon", "kaplan", "yapıyor", "yapma", "evde"]
words = "seni gördügümde gercekten mutlu oluyorum. inmazsan sor".split(" ")

# ✅ Convert words to IPA
ipa_results = {word: turkish_to_ipa(word) for word in words}
loanwords = {
    "telefon", "televizyon", "müzik", "kültür", "problem",
    "futbol", "doktor", "radyo", "otel", "tren", "klasik", "teknoloji"
}

# ✅ Process IPA Results with Fixes
corrected_ipa_results = {}
for word, ipa in ipa_results.items():
    if word in loanwords:
        # Ensure first occurrence of 'ʏ' → 'y' and 'ɛ' → 'e' are replaced
        if "ʏ" in ipa:
            ipa = ipa.replace("ʏ","y")
        if "ɛ" in ipa:
            ipa = ipa.replace("ɛ","e")
    corrected_ipa_results[word] = ipa

# ✅ Display Results
new_sen=""
for word, ipa in corrected_ipa_results.items():
    new_sen += ipa + " "
    
    print(f"{word} → {ipa}")
print(new_sen.replace("/",""))

seni → /sɛ.ˈni/
gördügümde → /ɡøɾ.dy.ɡym.ˈdɛ/
gercekten → /ɡɛɾ.dʒɛk.ˈtɛn/
mutlu → /mut.ˈɫu/
oluyorum. → /o.ɫu.jo.ˈɾum./
inmazsan → /in.maz.ˈsan/
sor → /soɾ/
sɛ.ˈni ɡøɾ.dy.ɡym.ˈdɛ ɡɛɾ.dʒɛk.ˈtɛn mut.ˈɫu o.ɫu.jo.ˈɾum. in.maz.ˈsan soɾ 
